<a href="https://colab.research.google.com/github/marinasjp/Federated-Learning/blob/main/FederatedLearningTest1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install --quiet --upgrade tensorflow-federated
!pip install --quiet --upgrade tensorflow-model-optimization
!pip install --quiet --upgrade nest-asyncio

import nest_asyncio
nest_asyncio.apply()

%load_ext tensorboard

!pip install tensorflow
!pip install --quiet tensorflow-federated==0.20.0

import pandas as pd
import collections
import numpy as np
np.random.seed(0)
from typing import Callable, Sequence, Tuple, Union

import numpy as np

import tensorflow as tf

import tensorflow_federated as tff

from matplotlib import pyplot as plt

import tensorflow as tf
import tensorflow_datasets
from tensorflow.keras import datasets, layers, models

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
ERROR: Ignored the following versions that require a different python version: 0.34.0 Requires-Python ~=3.9.0; 0.36.0 Requires-Python ~=3.9.0
ERROR: Could not find a version that satisfies the requirement jaxlib~=0.1.76 (from tensorflow-federated) (from versions: 0.3.14, 0.3.15, 0.3.20, 0.3.22, 0.3.24, 0.3.25, 0.4.0, 0.4.1, 0.4.2, 0.4.3, 0.4.4, 0.4.6, 0.4.7, 0.4.9, 0.4.10, 0.4.11, 0.4.12, 0.4.13, 0.4.14, 0.4.15, 0.4.16, 0.4.17, 0.4.18, 0.4.19)
ERROR: No matching distribution found for jaxlib~=0.1.76


In [11]:
import tensorflow_datasets

In [6]:
# (train_images, train_labels), (test_images, test_labels) =datasets.cifar10.load_data()
# (train_images, train_labels), (test_images, test_labels) = datasets.cifar100.load_data()
# # Normalize pixel values to be between 0 and 1
# train_images, test_images = train_images / 255.0, test_images / 255.0


# train_ds = tf.data.Dataset.from_tensor_slices(
#     (train_images, train_labels)).shuffle(10000).batch(32)

# test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(32)

169001437/169001437 [==============================] - 2s 0us/step


In [ ]:
def dirichlet_split(train_idcs, train_labels, alpha, n_clients):
   '''
    Splits a list of data indices with corresponding labels
    into subsets according to a dirichlet distribution with parameter
    alpha
    '''
    n_classes = train_labels.max()+1
    label_distribution = np.random.dirichlet([alpha]*n_clients, n_classes)

    class_idcs = [np.argwhere(train_labels[train_idcs]==y).flatten()
           for y in range(n_classes)]

    client_idcs = [[] for _ in range(n_clients)]
    for c, fracs in zip(class_idcs, label_distribution):
        for i, idcs in enumerate(np.split(c, (np.cumsum(fracs)[:-1]*len(c)).astype(int))):
            client_idcs[i] += [idcs]

    client_idcs = [train_idcs[np.concatenate(idcs)] for idcs in client_idcs]

    return client_idcs


In [16]:
CIFAR_SHAPE = (32, 32, 3)
TOTAL_FEATURE_SIZE = 32 * 32 * 3
NUM_CLASSES = 10
TRAIN_EXAMPLES = 50000
TEST_EXAMPLES = 10000
# Number of training examples per class: 50,000 / 10.
TRAIN_EXAMPLES_PER_LABEL = 5000
# Number of test examples per class: 10,000 / 10.
TEST_EXAMPLES_PER_LABEL = 1000


def load_cifar10_federated(
    dirichlet_parameter: float = 1,
    num_clients: int = 10,
) -> Tuple[tff.simulation.datasets.ClientData,
           tff.simulation.datasets.ClientData]:
  """Construct a federated dataset from the centralized CIFAR-10.

  Sampling based on Dirichlet distribution over categories, following the paper
  Measuring the Effects of Non-Identical Data Distribution for
  Federated Visual Classification (https://arxiv.org/abs/1909.06335).

  Args:
    dirichlet_parameter: Parameter of Dirichlet distribution. Each client
      samples from this Dirichlet to get a multinomial distribution over
      classes. It controls the data heterogeneity of clients. If approaches 0,
      then each client only have data from a single category label. If
      approaches infinity, then the client distribution will approach IID
      partitioning.
    num_clients: The number of clients the examples are going to be partitioned
      on.

  Returns:
    A tuple of `tff.simulation.datasets.ClientData` representing unpreprocessed
    train data and test data.
  """
  train_images, train_labels = tensorflow_datasets.as_numpy(
      tensorflow_datasets.load(
          name='cifar10',
          split='train',
          batch_size=-1,
          as_supervised=True,
      ))
  test_images, test_labels = tensorflow_datasets.as_numpy(
      tensorflow_datasets.load(
          name='cifar10',
          split='test',
          batch_size=-1,
          as_supervised=True,
      ))
  train_clients = collections.OrderedDict()
  test_clients = collections.OrderedDict()

  train_multinomial_vals = []
  test_multinomial_vals = []
  # Each client has a multinomial distribution over classes drawn from a
  # Dirichlet.
  for i in range(num_clients):
    proportion = np.random.dirichlet(dirichlet_parameter *
                                     np.ones(NUM_CLASSES,))
    train_multinomial_vals.append(proportion)
    test_multinomial_vals.append(proportion)

  train_multinomial_vals = np.array(train_multinomial_vals)
  test_multinomial_vals = np.array(test_multinomial_vals)

  train_example_indices = []
  test_indices = []
  for k in range(NUM_CLASSES):
    train_label_k = np.where(train_labels == k)[0]
    np.random.shuffle(train_label_k)
    train_example_indices.append(train_label_k)
    test_label_k = np.where(test_labels == k)[0]
    np.random.shuffle(test_label_k)
    test_indices.append(test_label_k)

  train_example_indices = np.array(train_example_indices)
  test_indices = np.array(test_indices)

  train_client_samples = [[] for _ in range(num_clients)]
  test_client_samples = [[] for _ in range(num_clients)]
  train_count = np.zeros(NUM_CLASSES).astype(int)
  test_count = np.zeros(NUM_CLASSES).astype(int)

  train_examples_per_client = int(TRAIN_EXAMPLES / num_clients)
  test_examples_per_client = int(TEST_EXAMPLES / num_clients)
  for k in range(num_clients):

    for i in range(train_examples_per_client):
      sampled_label = np.argwhere(
          np.random.multinomial(1, train_multinomial_vals[k, :]) == 1)[0][0]
      train_client_samples[k].append(
          train_example_indices[sampled_label, train_count[sampled_label]])
      train_count[sampled_label] += 1
      if train_count[sampled_label] == TRAIN_EXAMPLES_PER_LABEL:
        train_multinomial_vals[:, sampled_label] = 0
        train_multinomial_vals = (
            train_multinomial_vals /
            train_multinomial_vals.sum(axis=1)[:, None])

    for i in range(test_examples_per_client):
      sampled_label = np.argwhere(
          np.random.multinomial(1, test_multinomial_vals[k, :]) == 1)[0][0]
      test_client_samples[k].append(test_indices[sampled_label,
                                                 test_count[sampled_label]])
      test_count[sampled_label] += 1
      if test_count[sampled_label] == TEST_EXAMPLES_PER_LABEL:
        test_multinomial_vals[:, sampled_label] = 0
        test_multinomial_vals = (
            test_multinomial_vals / test_multinomial_vals.sum(axis=1)[:, None])

  for i in range(num_clients):
    client_name = str(i)
    x_train = train_images[np.array(train_client_samples[i])]
    y_train = train_labels[np.array(
        train_client_samples[i])].astype('int64').squeeze()
    train_data = collections.OrderedDict(
        (('image', x_train), ('label', y_train)))
    train_clients[client_name] = train_data

    x_test = test_images[np.array(test_client_samples[i])]
    y_test = test_labels[np.array(
        test_client_samples[i])].astype('int64').squeeze()
    test_data = collections.OrderedDict((('image', x_test), ('label', y_test)))
    test_clients[client_name] = test_data

  train_dataset = tff.simulation.datasets.ClientData(train_clients)
  test_dataset = tff.simulation.datasets.ClientData(test_clients)

  return train_dataset, test_dataset


train_dataset, test_dataset = load_cifar10_federated()

<ipython-input-16-0458a0ac0ed6>:98: RuntimeWarning: invalid value encountered in divide
  train_multinomial_vals /
<ipython-input-16-0458a0ac0ed6>:110: RuntimeWarning: invalid value encountered in divide
  test_multinomial_vals / test_multinomial_vals.sum(axis=1)[:, None])


In [14]:
# print(train_dataset)

In [ ]:
BATCH_SIZE = 32
num_classes = 10
EPOCHS = 15
NUM_CLIENTS = 100

# client_datasets = []
# for i in range(NUM_CLIENTS):
#     # Randomly sample a subset of the dataset.
#     client_dataset_x = train_images[tf.random.shuffle(tf.range(len(train_images))), :]
#     client_dataset_y = train_labels[tf.random.shuffle(tf.range(len(train_labels))), :]

#     # Create a TensorFlow dataset for the client.
#     client_dataset = emnist_train.create_tf_dataset_for_client(emnist_train.client_ids[i])
#     # client_dataset = tf.data.Dataset.from_tensor_slices((client_dataset_x, client_dataset_y))

#     # Add the client dataset to the list of client datasets.
#     client_datasets.append(client_dataset)


# def preprocess(dataset):

#   def batch_format_fn(element):
#     """Flatten a batch of EMNIST data and return a (features, label) tuple."""
#     return (tf.reshape(element['pixels'], [-1, 784]),
#             tf.reshape(element['label'], [-1, 1]))

#   return dataset.batch(BATCH_SIZE).map(batch_format_fn)

# client_ids = sorted(train_images.client_ids)[:NUM_CLIENTS]
# federated_train_data = [preprocess(train_images.create_tf_dataset_for_client(x))
#   for x in client_ids
# ]

In [ ]:
def create_keras_model():
  initializer = tf.keras.initializers.GlorotNormal(seed=0)
  return tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(32, 3, padding='same', input_shape=train_images.shape[1:], activation='relu'),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.25),

    # tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    # tf.keras.layers.Conv2D(64, 3, activation='relu'),
    # tf.keras.layers.MaxPooling2D(),
    # tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Conv2D(512, 3, padding='same', activation='relu'),
    tf.keras.layers.Conv2D(512, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.48),
    tf.keras.layers.Dense(num_classes, activation='softmax'),
  ])


def model_fn():
  keras_model = create_keras_model()
  return tff.learning.models.from_keras_model(
      keras_model,
      input_spec=federated_train_data[0].element_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])


whimsy_model = model_fn()
@tff.tf_computation
def server_init():
  model = model_fn()
  return model.trainable_variables
@tff.federated_computation
def initialize_fn():
  return tff.federated_value(server_init(), tff.SERVER)

tf_dataset_type = tff.SequenceType(whimsy_model.input_spec)

model_weights_type = server_init.type_signature.result


In [ ]:

@tf.function
def client_update(model, dataset, server_weights, client_optimizer):
  """Performs training (using the server model weights) on the client's dataset."""
  # Initialize the client model with the current server weights.
  client_weights = model.trainable_variables
  # Assign the server weights to the client model.
  tf.nest.map_structure(lambda x, y: x.assign(y),
                        client_weights, server_weights)

  # Use the client_optimizer to update the local model.
  for batch in dataset:
    with tf.GradientTape() as tape:
      # Compute a forward pass on the batch of data
      outputs = model.forward_pass(batch)

    # Compute the corresponding gradient
    grads = tape.gradient(outputs.loss, client_weights)
    grads_and_vars = zip(grads, client_weights)

    # Apply the gradient using a client optimizer.
    client_optimizer.apply_gradients(grads_and_vars)

  return client_weights

# tff.tf_computation accepts a client datasets and server weights, and outputs
# an updated client weights tensor.
@tff.tf_computation(tf_dataset_type, model_weights_type)
def client_update_fn(tf_dataset, server_weights):
  model = model_fn()
  client_optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
  return client_update(model, tf_dataset, server_weights, client_optimizer)

In [ ]:

@tf.function
def server_update(model, mean_client_weights):
  """Updates the server model weights as the average of the client model weights."""
  model_weights = model.trainable_variables
  # Assign the mean client weights to the server model.
  tf.nest.map_structure(lambda x, y: x.assign(y),
                        model_weights, mean_client_weights)
  return model_weights

@tff.tf_computation(model_weights_type)
def server_update_fn(mean_client_weights):
  model = model_fn()
  return server_update(model, mean_client_weights)


In [ ]:
federated_server_type = tff.FederatedType(model_weights_type, tff.SERVER)
federated_dataset_type = tff.FederatedType(tf_dataset_type, tff.CLIENTS)

@tff.tf_computation
def server_init():
  model = model_fn()
  return model.trainable_variables


@tff.federated_computation
def initialize_fn():
  return tff.federated_value(server_init(), tff.SERVER)


In [ ]:
@tff.federated_computation(federated_server_type, federated_dataset_type)
def next_fn(server_weights, federated_dataset):
  # Broadcast the server weights to the clients.
  server_weights_at_client = tff.federated_broadcast(server_weights)

  # Each client computes their updated weights.
  client_weights = tff.federated_map(client_update_fn, (federated_dataset, server_weights_at_client))

  # The server averages these updates.
  mean_client_weights = tff.federated_mean(client_weights)

  # The server updates its model.
  server_weights = tff.federated_map(server_update_fn, mean_client_weights)

  return server_weights


In [ ]:

federated_algorithm = tff.templates.IterativeProcess(
    initialize_fn=initialize_fn,
    next_fn=next_fn
)

central_emnist_test = emnist_test.create_tf_dataset_from_all_clients()
central_emnist_test = preprocess(central_emnist_test)

def evaluate(server_state):
  keras_model = create_keras_model()
  keras_model.compile(
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
  )
  keras_model.set_weights(server_state)
  keras_model.evaluate(central_emnist_test)

server_state = federated_algorithm.initialize()
evaluate(server_state)

NUM_ROUNDS = 15
for round in range(NUM_ROUNDS):
  server_state = federated_algorithm.next(server_state, federated_train_data)

evaluate(server_state)


In [ ]:

# def split_image_data_realwd(data, labels, n_clients=100, verbose=True):
#   '''
#   Splits (data, labels) among 'n_clients s.t. every client can holds any number of classes which is trying to simulate real world dataset
#   Input:
#     data : [n_data x shape]
#     labels : [n_data (x 1)] from 0 to n_labels(10)
#     n_clients : number of clients
#     verbose : True/False => True for printing some info, False otherwise
#   Output:
#     clients_split : splitted client data into desired format
#   '''
#   def break_into(n,m):
#     '''
#     return m random integers with sum equal to n
#     '''
#     to_ret = [1 for i in range(m)]
#     for i in range(n-m):
#         ind = random.randint(0,m-1)
#         to_ret[ind] += 1
#     return to_ret

#   #### constants ####
#   n_classes = len(set(labels))
#   classes = list(range(n_classes))
#   np.random.shuffle(classes)
#   label_indcs  = [list(np.where(labels==class_)[0]) for class_ in classes]

#   #### classes for each client ####
#   tmp = [np.random.randint(1,10) for i in range(n_clients)]
#   total_partition = sum(tmp)

#   #### create partition among classes to fulfill criteria for clients ####
#   class_partition = break_into(total_partition, len(classes))

#   #### applying greedy approach first come and first serve ####
#   class_partition = sorted(class_partition,reverse=True)
#   class_partition_split = {}

#   #### based on class partition, partitioning the label indexes ###
#   for ind, class_ in enumerate(classes):
#       class_partition_split[class_] = [list(i) for i in np.array_split(label_indcs[ind],class_partition[ind])]

# #   print([len(class_partition_split[key]) for key in  class_partition_split.keys()])

#   clients_split = []
#   count = 0
#   for i in range(n_clients):
#     n = tmp[i]
#     j = 0
#     indcs = []

#     while n>0:
#         class_ = classes[j]
#         if len(class_partition_split[class_])>0:
#             indcs.extend(class_partition_split[class_][-1])
#             count+=len(class_partition_split[class_][-1])
#             class_partition_split[class_].pop()
#             n-=1
#         j+=1

#     ##### sorting classes based on the number of examples it has #####
#     classes = sorted(classes,key=lambda x:len(class_partition_split[x]),reverse=True)
#     if n>0:
#         raise ValueError(" Unable to fulfill the criteria ")
#     clients_split.append([data[indcs], labels[indcs]])
# #   print(class_partition_split)
# #   print("total example ",count)


#   def print_split(clients_split):
#     print("Data split:")
#     for i, client in enumerate(clients_split):
#       split = np.sum(client[1].reshape(1,-1)==np.arange(n_labels).reshape(-1,1), axis=1)
#       print(" - Client {}: {}".format(i,split))
#     print()

#     if verbose:
#       print_split(clients_split)

#   clients_split = np.array(clients_split)

#   return clients_split



In [ ]:


# def split_image_data(data, labels, n_clients=100, classes_per_client=10, shuffle=True, verbose=True):
#   '''
#   Splits (data, labels) among 'n_clients s.t. every client can holds 'classes_per_client' number of classes
#   Input:
#     data : [n_data x shape]
#     labels : [n_data (x 1)] from 0 to n_labels
#     n_clients : number of clients
#     classes_per_client : number of classes per client
#     shuffle : True/False => True for shuffling the dataset, False otherwise
#     verbose : True/False => True for printing some info, False otherwise
#   Output:
#     clients_split : client data into desired format
#   '''
#   #### constants ####
#   n_data = data.shape[0]
#   n_labels = np.max(labels) + 1


#   ### client distribution ####
#   data_per_client = clients_rand(len(data), n_clients)
#   data_per_client_per_class = [np.maximum(1,nd // classes_per_client) for nd in data_per_client]

#   # sort for labels
#   data_idcs = [[] for i in range(n_labels)]
#   for j, label in enumerate(labels):
#     data_idcs[label] += [j]
#   if shuffle:
#     for idcs in data_idcs:
#       np.random.shuffle(idcs)

#   # split data among clients
#   clients_split = []
#   c = 0
#   for i in range(n_clients):
#     client_idcs = []

#     budget = data_per_client[i]
#     c = np.random.randint(n_labels)
#     while budget > 0:
#       take = min(data_per_client_per_class[i], len(data_idcs[c]), budget)

#       client_idcs += data_idcs[c][:take]
#       data_idcs[c] = data_idcs[c][take:]

#       budget -= take
#       c = (c + 1) % n_labels

#     clients_split += [(data[client_idcs], labels[client_idcs])]

#   def print_split(clients_split):
#     print("Data split:")
#     for i, client in enumerate(clients_split):
#       split = np.sum(client[1].reshape(1,-1)==np.arange(n_labels).reshape(-1,1), axis=1)
#       print(" - Client {}: {}".format(i,split))
#     print()

#     if verbose:
#       print_split(clients_split)

#   clients_split = np.array(clients_split)

#   return clients_split
